# Linge & Langtagen, "Programming for Computations"
## Ch. 3.7 Double and triple integrals

### 3.7.1 The midpoint rule for a double integral

Given a double integral over a rectangular domain $[a,b]\times [c,d]$,
$$
\int_a^b \int_c^d f(x,y) dydx,
$$
how can we approximate this integral by numerical methods?

**Derivation via one-dimensional integrals**

Since we know how to deal with integrals in one variable, 
a fruitful approach is to view the double integral as two integrals, each in one variable, 
which can be approximated numerically by previous one-dimensional formulas. 
To this end, we introduce a help function $g(x)$ and write

$$
\int_a^b \int_c^d f(x,y) dydx = \int_a^b g(x)dx,
\quad g(x) = \int_c^d f(x,y) dy\thinspace .
$$

Each of the integrals

$$
\int_a^b g(x)dx,\quad g(x) = \int_c^d f(x,y) dy
$$

can be discretized by any numerical integration rule for an integral in one variable.
Let us use the midpoint method (3.21) and start with $g(x)=\int_c^d f(x,y)dy$. 
We introduce $n_y$ intervals on $[c,d]$ with length $h_y$. 
The midpoint rule for this integral then becomes

$$
g(x) = \int_c^d f(x,y) dy \approx  h_y \sum_{j=0}^{n_y-1} f(x,y_j),
\quad y_j = c + \frac{1}{2}{h_y} + jh_y .
$$

The expression looks somewhat different from (3.21), but that is because of the notation: since we integrate in the $y$ direction and will have to work with both $x$ and $y$ as coordinates, we must use $n_y$ for $n$, $h_y$ for $h$, and the counter $i$ is more naturally called $j$ when integrating in $y$. Integrals in the $x$ direction will use $h_x$ and $n_x$ for $h$ and $n$, and $i$ as counter.

The double integral is $\int_a^b g(x)dx$, which can be approximated by the midpoint method:

$$
\int_a^b g(x)dx \approx h_x \sum_{i=0}^{n_x-1} g(x_i),\quad x_i=a + \frac{1}{2}{h_x} + ih_x\thinspace .
$$
    
Putting the formulas together, we arrive at the *composite midpoint method for a double integral*:

$$
\begin{align}
\int_a^b \int_c^d f(x,y) dydx 
&\approx
h_x \sum_{i=0}^{n_x-1} h_y \sum_{j=0}^{n_y-1} f(x_i,y_j)\nonumber\\ 
&=
h_x h_y \sum_{i=0}^{n_x-1} \sum_{j=0}^{n_y-1} f(a + \frac{h_x}{2} + ih_x, c + \frac{h_y}{2} + jh_y)\thinspace .
\tag{3.25}
\end{align}
$$

**Direct derivation**

The formula (3.25) can also be derived directly in the two-dimensional case by applying the idea of the midpoint method. We divide the rectangle $[a,b]\times[c,d]$ into $n_x \times n_y$ equal-sized cells. The idea of the midpoint method is to approximate f by a constant over each cell, and evaluate the constant at the midpoint. Cell $(i,j)$ occupies the area

$$
[a+ih_x,a+(i+1)h_x]\times [c+jh_y, c+ (j+1)h_y],
$$

and the midpoint is  $(x_i,y_i)$ with

$$
x_i=a + ih_x + \frac{1}{2}{h_x} ,\quad y_j = c + jh_y + \frac{1}{2}{h_y}.
$$

The integral over the cell is therefore $h_xh_y f(x_i,y_j)$, and the total double integral is the sum over all cells, which is nothing but formula (3.25).

**Programming a double sum**

The formula (3.25) involves a double sum, which is normally implemented as a double for loop. A Python function implementing (3.25) may look like

we can compute some integral, e.g., $\int_2^3 \int_0^2 (2x+y)dxdy=9$ in an interactive shell and demonstrate that the function computes the right number:

from midpoint_double import midpoint_double1

def f(x, y):
   return 2*x + y

midpoint_double1(f, 0, 2, 2, 3, 5, 5)

In [ ]:
def midpoint_double1(f, a, b, c, d, nx, ny):
    hx = (b - a)/float(nx)
    hy = (d - c)/float(ny)
    I = 0
    for i in range(nx):
        for j in range(ny):
            xi = a + hx/2 + i*hx
            yj = c + hy/2 + j*hy
            I += hx*hy*f(xi, yj)
    return I

def f(x, y):
    return 2*x + y

p = midpoint_double1(f, 0, 2, 2, 3, 5, 5)

print(p)

**Reusing code for one-dimensional integrals**

It is very natural to write a two-dimensional midpoint method as we did in function midpoint_double1 when we have the formula (3.25). However, we could alternatively ask, much as we did in the mathematics, can we reuse a well-tested implementation for one-dimensional integrals to compute double integrals? That is, can we use function midpoint

def midpoint(f, a, b, n):
    h = float(b-a)/n
    result = 0
    for i in range(n):
        result += f((a + h/2.0) + i*h)
    result *= h
    return result

from the Section 3.3.2 "twice"? The answer is yes, if we think as we did in the mathematics: compute the double integral as a midpoint rule for integrating $g(x)$ and define $g(x_i)$ in terms of a midpoint rule over $f$ in the $y$ coordinate. The corresponding function has very short code:

def midpoint_double2(f, a, b, c, d, nx, ny):
    def g(x):
        return midpoint(lambda y: f(x, y), c, d, ny)

    return midpoint(g, a, b, nx)

The important advantage of this implementation is that we reuse a well-tested function for the standard one-dimensional midpoint rule and that we apply the one-dimensional rule exactly as in the mathematics.

**Verification via test functions**

How can we test that our functions for the double integral work?
The midpoint rule is exact for linear functions, regardless of how many subinterval we use. Also, any linear two-dimensional function $f(x,y)=px+qy+r$ will be integrated exactly by the two-dimensional midpoint rule. We may pick $f(x,y)=2x+y$ and create a proper test function that can automatically verify our two alternative implementations of the two-dimensional midpoint rule. To compute the integral of $f(x,y)$ we take advantage of SymPy to eliminate the possibility of errors in hand calculations. The test function becomes

In [ ]:
def midpoint_double1(f, a, b, c, d, nx, ny):
    hx = (b - a)/float(nx)
    hy = (d - c)/float(ny)
    I = 0
    for i in range(nx):
        for j in range(ny):
            xi = a + hx/2 + i*hx
            yj = c + hy/2 + j*hy
            I += hx*hy*f(xi, yj)
    return I

def midpoint(f, a, b, n):
    h = float(b-a)/n
    result = 0
    for i in range(n):
        result += f((a + h/2.0) + i*h)
    result *= h
    return result

def midpoint_double2(f, a, b, c, d, nx, ny):
    def g(x):
        return midpoint(lambda y: f(x, y), c, d, ny)

    return midpoint(g, a, b, nx)




def test_midpoint_double():
    """Test that a linear function is integrated exactly."""
    def f(x, y):
        return 2*x + y

    a = 0;  b = 2;  c = 2;  d = 3
    import sympy
    x, y = sympy.symbols('x  y')
    I_expected = sympy.integrate(f(x, y), (x, a, b), (y, c, d))
    # Test three cases: nx < ny, nx = ny, nx > ny
    for nx, ny in (3, 5), (4, 4), (5, 3):
        I_computed1 = midpoint_double1(f, a, b, c, d, nx, ny)
        I_computed2 = midpoint_double2(f, a, b, c, d, nx, ny)
        tol = 1E-14
        #print I_expected, I_computed1, I_computed2
        print(abs(I_computed1 - I_expected) < tol)
        print(abs(I_computed1 - I_expected) < tol)

if __name__ == '__main__':
    print(test_midpoint_double())

### 3.7.2 The midpoint rule for a triple integral

**Theory**

Once a method that works for a one-dimensional problem is generalized to two dimensions, it is usually quite straightforward to extend the method to three dimensions. This will now be demonstrated for integrals. We have the triple integral

$$
\int_{a}^{b} \int_c^d \int_e^f g(x,y,z) dzdydx
$$

and want to and want to approximate the integral by a midpoint rule. Following the ideas for the double integral, we split this integral into one-dimensional integrals:
    
$$
\begin{align*}
p(x,y) &= \int_e^f g(x,y,z) dz\\ 
q(x) &= \int_c^d p(x,y) dy\\ 
\int_{a}^{b} \int_c^d \int_e^f g(x,y,z) dzdydx &= \int_a^b q(x)dx
\end{align*}
$$

For each of these one-dimensional integrals we apply the midpoint rule:
    
$$
\begin{align*}
p(x,y) = \int_e^f g(x,y,z) dz
&\approx \sum_{k=0}^{n_z-1} g(x,y,z_k),\\ 
q(x) = \int_c^d p(x,y) dy
&\approx \sum_{j=0}^{n_y-1} p(x,y_j),\\ 
\int_{a}^{b} \int_c^d \int_e^f g(x,y,z) dzdydx = \int_a^b q(x)dx
&\approx \sum_{i=0}^{n_x-1} q(x_i),
\end{align*}
$$

where

$$
z_k=e + \frac{1}{2}h_z + kh_z,\quad y_j=c + \frac{1}{2}h_y + jh_y \quad
x_i=a + \frac{1}{2}h_x + ih_x.
$$

Starting with the formula for $\int_{a}^{b} \int_c^d \int_e^f g(x,y,z) dzdydx$ and inserting the two previous formulas gives

$$
\begin{align}
& \int_{a}^{b} \int_c^d \int_e^f g(x,y,z)\, dzdydx\approx\nonumber\\ 
& h_xh_yh_z
\sum_{i=0}^{n_x-1}\sum_{j=0}^{n_y-1}\sum_{k=0}^{n_z-1}
g(a + \frac{1}{2}h_x + ih_x,
c + \frac{1}{2}h_y + jh_y,
e + \frac{1}{2}h_z + kh_z)\thinspace .
\tag{3.26}
\end{align}
$$



**Implementation**

We follow the ideas for the implementations of the midpoint rule for a double integral. The corresponding functions are shown below and found in the file midpoint_triple.py.

In [ ]:
def midpoint_triple1(g, a, b, c, d, e, f, nx, ny, nz):
    hx = (b - a)/float(nx)
    hy = (d - c)/float(ny)
    hz = (f - e)/float(nz)
    I = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                xi = a + hx/2 + i*hx
                yj = c + hy/2 + j*hy
                zk = e + hz/2 + k*hz
                I += hx*hy*hz*g(xi, yj, zk)
    return I

def midpoint(f, a, b, n):
    h = float(b-a)/n
    result = 0
    for i in range(n):
        result += f((a + h/2.0) + i*h)
    result *= h
    return result

def midpoint_triple2(g, a, b, c, d, e, f, nx, ny, nz):
    def p(x, y):
        return midpoint(lambda z: g(x, y, z), e, f, nz)

    def q(x):
        return midpoint(lambda y: p(x, y), c, d, ny)

    return midpoint(q, a, b, nx)

def test_midpoint_triple():
    """Test that a linear function is integrated exactly."""
    def g(x, y, z):
        return 2*x + y - 4*z

    a = 0;  b = 2;  c = 2;  d = 3;  e = -1;  f = 2
    import sympy
    x, y, z = sympy.symbols('x y z')
    I_expected = sympy.integrate(
        g(x, y, z), (x, a, b), (y, c, d), (z, e, f))
    for nx, ny, nz in (3, 5, 2), (4, 4, 4), (5, 3, 6):
        I_computed1 = midpoint_triple1(
            g, a, b, c, d, e, f, nx, ny, nz)
        I_computed2 = midpoint_triple2(
            g, a, b, c, d, e, f, nx, ny, nz)
        tol = 1E-14
        print(I_expected, I_computed1, I_computed2)
        assert abs(I_computed1 - I_expected) < tol
        assert abs(I_computed2 - I_expected) < tol

if __name__ == '__main__':
    test_midpoint_triple()